<a href="https://colab.research.google.com/github/2j0123/WASSUP_EST_FINAL_Team4/blob/SW/Inception-V3-epoch%20170.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Drive와 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 필요한 라이브러리 import
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# 데이터 경로 설정
base_dir = '/content/drive/MyDrive/감정따뜻쟁이/train'

# 이미지 데이터 생성기 설정
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# 이미지 데이터 로드
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

# Inception-v3 모델 불러오기 (사전 훈련된 가중치 포함)
base_model = InceptionV3(weights='imagenet', include_top=False)

# 분류기 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

# 새로운 모델 정의 (기존의 Inception-v3 모델과 분류기를 결합)
model = Model(inputs=base_model.input, outputs=predictions)

# 모든 레이어를 동결하여 기존의 가중치를 고정
for layer in base_model.layers:
    layer.trainable = False

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit(train_generator, epochs=170)

# 새로운 이미지 분류 테스트
def classify_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(299, 299))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, 0)  # 이미지 배치 추가
    img_array /= 255.  # 이미지 정규화
    predictions = model.predict(img_array)
    emotion_labels = ['anger', 'happy', 'sadness', 'panic']
    predicted_label = emotion_labels[np.argmax(predictions)]
    return predicted_label

# 예시: 이미지 분류 테스트
test_image_path = '/content/drive/MyDrive/test_image.jpg'  # 테스트할 이미지 파일의 경로
predicted_label = classify_image(test_image_path)
print('Predicted emotion:', predicted_label)

Found 5369 images belonging to 4 classes.
87910968/87910968 [==============================] - 3s 0us/step
Epoch 1/170
168/168 [==============================] - 2498s 15s/step - loss: 1.5416 - accuracy: 0.3198
Epoch 2/170
168/168 [==============================] - 404s 2s/step - loss: 1.3103 - accuracy: 0.3762
Epoch 3/170
168/168 [==============================] - 402s 2s/step - loss: 1.2916 - accuracy: 0.3863
Epoch 4/170
168/168 [==============================] - 405s 2s/step - loss: 1.2655 - accuracy: 0.4081
Epoch 5/170
168/168 [==============================] - 398s 2s/step - loss: 1.2473 - accuracy: 0.4308
Epoch 6/170
168/168 [==============================] - 394s 2s/step - loss: 1.2456 - accuracy: 0.4267
Epoch 7/170
168/168 [==============================] - 397s 2s/step - loss: 1.2212 - accuracy: 0.4411
Epoch 8/170
168/168 [==============================] - 395s 2s/step - loss: 1.2096 - accuracy: 0.4425
Epoch 9/170
168/168 [==============================] - 410s 2s/step - loss:

In [3]:
from google.colab import files
from google.colab.patches import cv2_imshow
import cv2

# 동영상 처리 함수
def classify_video(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # 프레임을 저장하고 분류
        frame_path = 'frame.jpg'
        cv2.imwrite(frame_path, frame)
        predicted_label = classify_image(frame_path)
        print(f'Predicted emotion for current frame: {predicted_label}')
        cv2_imshow(frame)
    cap.release()

# 사용자가 원하는 만큼 사진 또는 동영상을 업로드하고 분류
while True:
    uploaded = files.upload()
    for filename in uploaded.keys():
        file_path = '/content/' + filename
        if filename.lower().endswith(('jpg', 'jpeg', 'png')):
            predicted_label = classify_image(file_path)
            print(f'{filename}의 감정은 {predicted_label}입니다.')
        elif filename.lower().endswith(('mp4', 'avi', 'mov', 'mkv')):
            classify_video(file_path)
        else:
            print(f'{filename}은 지원하지 않는 파일 형식입니다.')

    continue_upload = input("더 업로드하시겠습니까? (yes/no): ")
    if continue_upload.lower() != 'yes':
        break

Saving di51422c1d02b44c3e72e2672480c42d26ddd21043adcb50f231cb1c43048x41j.jpg to di51422c1d02b44c3e72e2672480c42d26ddd21043adcb50f231cb1c43048x41j.jpg
1/1 [==============================] - 0s 30ms/step
di51422c1d02b44c3e72e2672480c42d26ddd21043adcb50f231cb1c43048x41j.jpg의 감정은 happy입니다.
더 업로드하시겠습니까? (yes/no): yes


Saving dbktda9fbbe90330e4f95d8be038b0697d143dfeee49bc922c3ff480e50c99639.jpg to dbktda9fbbe90330e4f95d8be038b0697d143dfeee49bc922c3ff480e50c99639.jpg
1/1 [==============================] - 0s 26ms/step
dbktda9fbbe90330e4f95d8be038b0697d143dfeee49bc922c3ff480e50c99639.jpg의 감정은 happy입니다.
더 업로드하시겠습니까? (yes/no): yes


Saving yh0sa8c345a700362c6fb37d4cf47c74789a82704351c230c2e7b442e9c179h5g.jpg to yh0sa8c345a700362c6fb37d4cf47c74789a82704351c230c2e7b442e9c179h5g.jpg
1/1 [==============================] - 0s 25ms/step
yh0sa8c345a700362c6fb37d4cf47c74789a82704351c230c2e7b442e9c179h5g.jpg의 감정은 happy입니다.
더 업로드하시겠습니까? (yes/no): yes


Saving hl203a559cfc19e62cec1e36cc2adc9ebfc33c3acd85bf5fcc99af124e353uipk.jpg to hl203a559cfc19e62cec1e36cc2adc9ebfc33c3acd85bf5fcc99af124e353uipk.jpg
1/1 [==============================] - 0s 26ms/step
hl203a559cfc19e62cec1e36cc2adc9ebfc33c3acd85bf5fcc99af124e353uipk.jpg의 감정은 happy입니다.
더 업로드하시겠습니까? (yes/no): yes


Saving 5xcx46830a35bfb8554eba811bf483285ff70a43b542405ad05ff4befff516fvr.jpg to 5xcx46830a35bfb8554eba811bf483285ff70a43b542405ad05ff4befff516fvr.jpg
1/1 [==============================] - 0s 26ms/step
5xcx46830a35bfb8554eba811bf483285ff70a43b542405ad05ff4befff516fvr.jpg의 감정은 sadness입니다.
더 업로드하시겠습니까? (yes/no): yes


Saving rrpib48c0d6cfe150b54d6e59f1e5767558d95c94856b7645f70635c8e8d6brqj.jpg to rrpib48c0d6cfe150b54d6e59f1e5767558d95c94856b7645f70635c8e8d6brqj.jpg
1/1 [==============================] - 0s 26ms/step
rrpib48c0d6cfe150b54d6e59f1e5767558d95c94856b7645f70635c8e8d6brqj.jpg의 감정은 panic입니다.
더 업로드하시겠습니까? (yes/no): yes


Saving 9trz1c97bda4f9caf0ac2d3bc1404141a4c5424304ceadcfc847c4fc7f46cziqf.jpg to 9trz1c97bda4f9caf0ac2d3bc1404141a4c5424304ceadcfc847c4fc7f46cziqf.jpg
1/1 [==============================] - 0s 27ms/step
9trz1c97bda4f9caf0ac2d3bc1404141a4c5424304ceadcfc847c4fc7f46cziqf.jpg의 감정은 panic입니다.
더 업로드하시겠습니까? (yes/no): yes


Saving a63u7751b5fd52b5bd4edb7dc9e52dddd98cf2e1afdc0d9d0751ffb5ba40cnf3c.jpg to a63u7751b5fd52b5bd4edb7dc9e52dddd98cf2e1afdc0d9d0751ffb5ba40cnf3c.jpg
1/1 [==============================] - 0s 26ms/step
a63u7751b5fd52b5bd4edb7dc9e52dddd98cf2e1afdc0d9d0751ffb5ba40cnf3c.jpg의 감정은 sadness입니다.
더 업로드하시겠습니까? (yes/no): yes


Saving fny354d54b9f62f7eb263d63cd3219ff5fd5e2e4bc2f223d6fb5f24ae62632efh.jpg to fny354d54b9f62f7eb263d63cd3219ff5fd5e2e4bc2f223d6fb5f24ae62632efh.jpg
1/1 [==============================] - 0s 25ms/step
fny354d54b9f62f7eb263d63cd3219ff5fd5e2e4bc2f223d6fb5f24ae62632efh.jpg의 감정은 anger입니다.
더 업로드하시겠습니까? (yes/no): no
